Use npgamma to compare HN Case 7 dose data

In [1]:
import dicom
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from npgamma import calc_gamma

In [3]:
dcm_ref = dicom.read_file("Case7_dose_AAA.dcm")
dcm_evl = dicom.read_file("Case7_dose_Dm.dcm")

In [4]:
def load_dose_from_dicom(dcm):
    pixel_reference = np.transpose(
        dcm.pixel_array, (2, 1, 0))
    dose = (
        pixel_reference * dcm.DoseGridScaling)
    
    return dose


dose_reference = load_dose_from_dicom(dcm_ref)
dose_evaluation = load_dose_from_dicom(dcm_evl)

In [36]:
print('AAA')
print(dose_reference.shape)  # np.shape(dose_reference)
print(dcm_ref.PixelSpacing)
dcm_ref.ImagePositionPatient

AAA
(192, 107, 135)
['2.5', '2.5']


['-244.4338517', '-178.5440886', '-216.25']

In [37]:
print('AXB Dm')
print(dose_evaluation.shape)  # Dm slightly larger array
print(dcm_evl.PixelSpacing)
dcm_evl.ImagePositionPatient

AXB Dm
(193, 108, 135)
['2.5', '2.5']


['-245.6838517', '-179.7940886', '-216.25']

Dose grids slightly different, causes problems

In [38]:
def load_xyz_from_dicom(dcm):
    resolution = np.array(
        dcm.PixelSpacing).astype(float)
    dx = resolution[0]
    
    x = (
        dcm.ImagePositionPatient[0] + 
        np.arange(0, dcm.Rows * dx, dx))
    
    dy = resolution[1]
    y = (
        dcm.ImagePositionPatient[1] + 
        np.arange(0, dcm.Columns * dy, dy))
    
    z = (
        np.array(dcm.GridFrameOffsetVector) + 
        dcm.ImagePositionPatient[2])
    
    return x, y, z


x_reference, y_reference, z_reference = load_xyz_from_dicom(dcm_ref)
x_evaluation, y_evaluation, z_evaluation = load_xyz_from_dicom(dcm_evl)

In [44]:
print(x_reference.min(), x_reference.max())

-244.4338517 20.5661483


In [45]:
print(x_evaluation.min(), x_evaluation.max())

-245.6838517 21.8161483


In [39]:
coords_reference = (
    x_reference, y_reference, z_reference)

coords_evaluation = (
    x_evaluation, y_evaluation, z_evaluation)

In [40]:
distance_threshold = 3
distance_step_size = distance_threshold / 10

dose_threshold = 0.03 * np.max(dose_reference)
lower_dose_cutoff = np.max(dose_reference) * 0.2
maximum_test_distance = distance_threshold * 2
max_concurrent_calc_points = 10000000

In [41]:
gamma = calc_gamma(
    coords_reference, dose_reference,
    coords_evaluation, dose_evaluation,
    distance_threshold, dose_threshold,
    lower_dose_cutoff=lower_dose_cutoff, 
    distance_step_size=distance_step_size,
    maximum_test_distance=maximum_test_distance,
    max_concurrent_calc_points=max_concurrent_calc_points)

Exception: Length of items in coords_reference does not match the shape of dose_reference